## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |


所有 Runnable 对象都显式描述输入和输出 Schema，以检查输入和输出格式：

- input_schema：从Runnable的结构自动生成的输入Pydantic模型
- output_schema：从Runnable的结构自动生成的输出Pydantic模型 


### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")
chain = prompt | model

#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [7]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [8]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [9]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [10]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'proper

In [ ]:
model.output_schema.schema() # by wy：当然 与chain的输出 是一样的

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [11]:
for s in chain.stream({"topic": "程序员，使用英文"}):
    print(s.content, end="", flush=True)

Sure! Here's a classic programmer joke for you:

Why do programmers prefer dark mode?

Because light attracts bugs!

### Invoke
使用 .invoke() 方法单次（同步）调用

In [12]:
chain.invoke({"topic": "程序员，使用英文"})

AIMessage(content="Sure! Here's a programming joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 21, 'total_tokens': 42, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-ce6b69ca-30f4-4663-a773-13af5dcebb82-0')

### Batch
使用 .batch() 方法（同步）批量调用

In [13]:
chain.batch([{"topic": "程序员，使用英文"}, {"topic": "产品经理，使用英文"}, {"topic": "测试经理，使用英文"}])

[AIMessage(content="Sure! Here's a programmer joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 21, 'total_tokens': 42, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-4368c55c-871a-415e-a57a-c762e4218ed2-0'),
 AIMessage(content="Sure! Here's a joke about product managers:\n\nWhy did the product manager bring a ladder to the meeting?\n\nBecause they wanted to reach new heights in their project!", response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 19, 'total_tokens': 51, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_8552ec53e1', 'finish_reason': 'stop', 'logprobs': Non

In [14]:
messages = chain.batch([{"topic": "程序员，使用英文"}, {"topic": "产品经理，使用英文"}, {"topic": "测试经理，使用英文"}])

In [12]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:

Sure! Here's a classic programmer joke for you:

Why do programmers prefer dark mode?

Because light attracts bugs!


笑话1:

Sure! Here's a joke about product managers:

Why did the product manager bring a ladder to the meeting?

Because they wanted to reach new heights in their project!


笑话2:

Sure! Here's a joke about a testing manager:

Why did the testing manager bring a ladder to work?

Because they heard the job had a lot of "high-level" requirements!




## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [15]:
async for s in chain.astream({"topic": "程序员，使用英文"}):
    print(s.content, end="", flush=True)

Sure! Here's a programmer joke for you:

Why do programmers prefer dark mode?

Because light attracts bugs!

### Async Invoke

In [16]:
await chain.ainvoke({"topic": "程序员，使用英文"})

AIMessage(content='Sure! Here’s a classic programmer joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 21, 'total_tokens': 44, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-9370a8b7-497c-4d6c-87d9-19e4f5e1fdce-0')

### Async Batch

In [15]:
await chain.abatch([{"topic": "程序员，使用英文"}, {"topic": "产品经理，使用英文"}, {"topic": "测试经理，使用英文"}])

[AIMessage(content="Sure! Here's a programmer joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 21, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-270640b5-870c-4715-a1de-eda712347a93-0'),
 AIMessage(content='Sure! Here’s a light-hearted joke about product managers:\n\nWhy did the product manager bring a ladder to the meeting?\n\nBecause they wanted to reach new heights in their project!', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 19, 'total_tokens': 54}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None}, id='run-f78b7cf0-73bb-4d56-a2f5-39fb93cd7a72-0'),
 AIMessage(content="Sure, here's a light-hearted joke about a testing manager:\n\nWhy did the testing manager bring a ladder to work?\